In [1]:
import cv2
import os
import pickle
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
def hog(img_gray, cell_size=8, block_size=2, bins=9):
    img = img_gray
    h, w = img.shape # 128, 64
    
    # gradient
    xkernel = np.array([[-1, 0, 1]])
    ykernel = np.array([[-1], [0], [1]])
    dx = cv2.filter2D(img, cv2.CV_32F, xkernel)
    dy = cv2.filter2D(img, cv2.CV_32F, ykernel)
    
    # histogram
    magnitude = np.sqrt(np.square(dx) + np.square(dy))
    orientation = np.arctan(np.divide(dy, dx+0.00001)) # radian
    orientation = np.degrees(orientation) # -90 -> 90
    orientation += 90 # 0 -> 180
    
    num_cell_x = w // cell_size # 8
    num_cell_y = h // cell_size # 16
    hist_tensor = np.zeros([num_cell_y, num_cell_x, bins]) # 16 x 8 x 9
    for cx in range(num_cell_x):
        for cy in range(num_cell_y):
            ori = orientation[cy*cell_size:cy*cell_size+cell_size, cx*cell_size:cx*cell_size+cell_size]
            mag = magnitude[cy*cell_size:cy*cell_size+cell_size, cx*cell_size:cx*cell_size+cell_size]
            # https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html
            hist, _ = np.histogram(ori, bins=bins, range=(0, 180), weights=mag) # 1-D vector, 9 elements
            hist_tensor[cy, cx, :] = hist
        pass
    pass
    
    # normalization
    redundant_cell = block_size-1
    feature_tensor = np.zeros([num_cell_y-redundant_cell, num_cell_x-redundant_cell, block_size*block_size*bins])
    for bx in range(num_cell_x-redundant_cell): # 7
        for by in range(num_cell_y-redundant_cell): # 15
            by_from = by
            by_to = by+block_size
            bx_from = bx
            bx_to = bx+block_size
            v = hist_tensor[by_from:by_to, bx_from:bx_to, :].flatten() # to 1-D array (vector)
            feature_tensor[by, bx, :] = v / LA.norm(v, 2)
            # avoid NaN:
            if np.isnan(feature_tensor[by, bx, :]).any(): # avoid NaN (zero division)
                feature_tensor[by, bx, :] = v
    
    return feature_tensor.flatten() # 3780 features

In [6]:
def extract_face(filename, required_size=(64, 128)):
    try:
        # Load image using OpenCV for better performance
        image = cv2.imread(filename)
        if image is None:
            print(f"Failed to load image: {filename}")
            return None

        # Convert to RGB format (OpenCV loads in BGR by default)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Create the MTCNN detector
        detector = MTCNN()

        # Detect faces in the image
        results = detector.detect_faces(image)
        if len(results) == 0:
            print(f"No face detected in {filename}")
            return None

        # Select the face with the highest confidence score
        best_result = max(results, key=lambda res: res['confidence'])

        # Extract the bounding box
        x1, y1, width, height = best_result['box']
        x1, y1 = max(0, x1), max(0, y1)  # Ensure coordinates are within bounds
        x2, y2 = x1 + width, y1 + height

        # Resize face to the model's required size
        face = image[y1:y2, x1:x2]

        # Convert the face region to grayscale for HOG
        face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Resize the face to the required size for HOG
        face_resized = cv2.resize(face_gray, required_size)

        # Compute the HOG features for the face
        hog_features = hog(face_resized)

        return hog_features
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return None

In [7]:
# Load dataset and extract features
def load_dataset(dataset_path):
    X, y = [], []

    for person_name in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_folder):
            continue

        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            
            hog_features = extract_face(img_path)
            if hog_features is not None:
                X.append(hog_features)
                y.append(person_name)
        print('Finish loading examples for class: %s' % person_name)
    return np.array(X), np.array(y)

In [6]:
# Load dataset
dataset_path = "dataset/"
X, y = load_dataset(dataset_path)

Finish loading examples for class: Andy Samberg
Finish loading examples for class: Billie Eilish
Finish loading examples for class: Brad Pitt
Finish loading examples for class: Camila Cabello
Finish loading examples for class: Charlize Theron
Finish loading examples for class: Claire Holt
Finish loading examples for class: Elizabeth Olsen
No face detected in dataset/Emma Stone\Emma Stone73_1817.jpg
No face detected in dataset/Emma Stone\Emma Stone86_1830.jpg
Finish loading examples for class: Emma Stone
Finish loading examples for class: Henry Cavill
Finish loading examples for class: Hugh Jackman
Finish loading examples for class: Jennifer Lawrence
Finish loading examples for class: Jessica Alba
Finish loading examples for class: Margot Robbie
Finish loading examples for class: Natalie Portman
Finish loading examples for class: Neil Patrick Harris
Finish loading examples for class: Robert Downey Jr
Finish loading examples for class: Roger Federer
Finish loading examples for class: Tom

In [7]:
with open("data.pkl", "wb") as file:
    pickle.dump((X, y), file)

print("Data saved")

Data saved


In [8]:
with open("data.pkl", "rb") as file:
    X, y = pickle.load(file)

In [9]:
# Train KNN
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train: ",X_train.shape)
print("X_test: ",X_test.shape)
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)

X_train:  (1573, 3780)
X_test:  (394, 3780)
y_train:  (1573,)
y_test:  (394,)


In [82]:
knn = KNeighborsClassifier(n_neighbors = 3, weights = "distance", algorithm = "ball_tree", leaf_size=10, metric="euclidean")

pca = PCA(n_components=150, svd_solver="randomized", whiten=True)
svm = SVC(C=500, gamma=0.005, class_weight='balanced', kernel='rbf', random_state=42)
model = make_pipeline(pca, svm)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [83]:
svm_train_acc = accuracy_score(y_train, model.predict(X_train))
svm_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of SVM is : {svm_train_acc}")
print(f"Test accuracy of SVM is : {svm_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of SVM is : 1.0
Test accuracy of SVM is : 0.7284263959390863
[[13  0  0  0  0  1  0  0  0  1  0  1  0  1  0  1  0  0  0  1]
 [ 0  6  0  0  0  2  0  1  0  0  0  1  1  1  2  0  0  0  0  0]
 [ 0  0 25  0  1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0]
 [ 0  0  1 17  2  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0]
 [ 0  0  0  0 11  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  1  0  1 14  0  0  1  0  2  0  3  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  1 13  0  0  0  0  5  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  1 14  1  0  3  1  1  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0 10  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  2 18  0  0  0  0  0  0  0  1  0  1]
 [ 0  2  0  1  1  2  0  1  0  0 24  0  0  2  0  1  0  0  0  0]
 [ 0  1  0  0  1  1  0  1  0  0  3 13  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 11  2  0  0  0  0  0  0]
 [ 0  0  0  0  2  1  0  0  1  0  0  0  1 19  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  1  2  0  0  1  